In [1]:
import datetime
import pandas as pd
import geopandas as gpd

import FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from utils import timed

In [2]:
tst = [2020, 9, 5, "AM"]
ted = [2020, 9, 8, "PM"]
region = ("Creek", [-119.5, 36.8, -118.9, 37.7])

list_of_ts = list(FireTime.t_generator(tst, ted))

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [ ]:
preprocess.preprocess_landcover()

## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [3]:
preprocess.preprocess_region(region)

2024-03-20 20:14:41,801 - FireLog - INFO - func:preprocess_region took: 675.35 ms


## Once per input file

Next process each monthly file into half day files. First we'll get all the times that are of interest. This could also be done by looking at all the files that exist and seeing which have not been preprocessed yet.

In [4]:
%%time
preprocess.preprocess_monthly_file([2020, 9], "SNPP")

2024-03-20 20:14:42,126 - FireLog - INFO - preprocessing monthly file for 2020-9, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:258: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-03-20 20:16:43,512 - FireLog - INFO - func:preprocess_monthly_file took: 2.02 min


CPU times: user 1min 11s, sys: 2.74 s, total: 1min 14s
Wall time: 2min 1s


Note: "NOAA20" files for this time period are not in the input folder.

## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

In [6]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="SNPP", region=region)

2024-03-20 20:16:52,541 - FireLog - INFO - filtering and clustering 2020-9-5 AM, SNPP, Creek
2024-03-20 20:16:52,565 - FireLog - INFO - func:read_preprocessed took: 22.83 ms
2024-03-20 20:16:52,567 - FireLog - INFO - func:read_region took: 1.36 ms
2024-03-20 20:16:52,567 - FireLog - INFO - func:preprocess_region_t took: 26.17 ms
2024-03-20 20:16:52,568 - FireLog - INFO - filtering and clustering 2020-9-5 PM, SNPP, Creek
2024-03-20 20:16:52,646 - FireLog - INFO - func:read_preprocessed took: 78.05 ms
2024-03-20 20:16:52,648 - FireLog - INFO - func:read_region took: 1.13 ms
2024-03-20 20:16:52,649 - FireLog - INFO - func:preprocess_region_t took: 81.39 ms
2024-03-20 20:16:52,650 - FireLog - INFO - filtering and clustering 2020-9-6 AM, SNPP, Creek
2024-03-20 20:16:52,676 - FireLog - INFO - func:read_preprocessed took: 26.23 ms
2024-03-20 20:16:52,678 - FireLog - INFO - func:read_region took: 1.15 ms
2024-03-20 20:16:52,679 - FireLog - INFO - func:preprocess_region_t took: 29.02 ms
2024-03

CPU times: user 409 ms, sys: 27.9 ms, total: 437 ms
Wall time: 446 ms


## Run the fire expansion and merging algorithm

`Fire_Forward` is responsible for reading in the preprocessed data created in the `Ingest` notebook

In [7]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, sat="SNPP", restart=False, region=region)

2024-03-20 20:16:53,007 - FireLog - INFO - func:read_preprocessed took: 4.66 ms
2024-03-20 20:16:53,013 - FireLog - INFO - func:read_preprocessed took: 5.88 ms
2024-03-20 20:16:53,025 - FireLog - INFO - func:read_preprocessed took: 11.28 ms
2024-03-20 20:16:53,031 - FireLog - INFO - func:read_preprocessed took: 5.61 ms
2024-03-20 20:16:53,042 - FireLog - INFO - func:read_preprocessed took: 9.65 ms
2024-03-20 20:16:53,049 - FireLog - INFO - func:read_preprocessed took: 7.28 ms
2024-03-20 20:16:53,063 - FireLog - INFO - func:read_preprocessed took: 13.26 ms
2024-03-20 20:16:53,076 - FireLog - INFO - func:read_preprocessed took: 12.72 ms
2024-03-20 20:16:53,080 - FireLog - INFO - --------------------
2024-03-20 20:16:53,080 - FireLog - INFO - Fire tracking at [2020, 9, 5, 'AM']
2024-03-20 20:16:53,114 - FireLog - INFO - func:Fire_expand_rtree took: 31.52 ms
2024-03-20 20:16:53,115 - FireLog - INFO - func:Fire_merge_rtree took: 0.75 ms
2024-03-20 20:16:53,117 - FireLog - INFO - func:invali

We can save these files off in case something goes wrong later

In [8]:
postprocess.save_allpixels(allpixels, tst, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, tst, ted, region)

2024-03-20 20:17:00,292 - FireLog - INFO - func:save_allpixels took: 156.64 ms
2024-03-20 20:17:00,353 - FireLog - INFO - func:save_allfires_gdf took: 59.46 ms


## Write Snapshot files

Start by reading in the outputs from the run:

In [3]:
allpixels = postprocess.read_allpixels(tst, ted, region)
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region)

2024-03-20 20:18:28,840 - FireLog - INFO - func:read_allpixels took: 31.09 ms
2024-03-20 20:18:28,950 - FireLog - INFO - func:read_allfires_gdf took: 109.88 ms


Then iterate through all of the ts and save the snapshot layers.

In [4]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-03-20 20:18:29,755 - FireLog - INFO - func:save_snapshot_layers took: 255.48 ms
2024-03-20 20:18:29,984 - FireLog - INFO - func:save_snapshot_layers took: 224.34 ms
2024-03-20 20:18:30,300 - FireLog - INFO - func:save_snapshot_layers took: 310.74 ms
2024-03-20 20:18:30,547 - FireLog - INFO - func:save_snapshot_layers took: 243.70 ms
2024-03-20 20:18:30,794 - FireLog - INFO - func:save_snapshot_layers took: 243.66 ms
2024-03-20 20:18:31,009 - FireLog - INFO - func:save_snapshot_layers took: 211.74 ms
2024-03-20 20:18:31,283 - FireLog - INFO - func:save_snapshot_layers took: 270.12 ms
2024-03-20 20:18:31,502 - FireLog - INFO - func:save_snapshot_layers took: 215.96 ms
2024-03-20 20:18:31,503 - FireLog - INFO - func:save_snapshots took: 2.01 sec


CPU times: user 886 ms, sys: 39.3 ms, total: 925 ms
Wall time: 2.01 s


## Write Largefire files

First we will find all the large fires:

In [5]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-03-20 20:18:31,513 - FireLog - INFO - func:find_largefires took: 4.62 ms


In [6]:
large_fires

array([1])

First we'll use the `allpixels` object to create the `nplist` layer

In [8]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

2024-03-20 20:18:37,779 - FireLog - INFO - func:save_fire_nplist took: 721.92 ms
2024-03-20 20:18:37,780 - FireLog - INFO - func:save_large_fires_nplist took: 725.37 ms


CPU times: user 600 ms, sys: 3.12 ms, total: 603 ms
Wall time: 726 ms


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [9]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst)

2024-03-20 20:18:39,155 - FireLog - INFO - func:merge_rows took: 16.89 ms


1 rows that potentially need a merge


2024-03-20 20:18:39,514 - FireLog - INFO - func:save_fire_layers took: 358.26 ms
2024-03-20 20:18:39,515 - FireLog - INFO - func:save_large_fires_layers took: 382.76 ms


CPU times: user 232 ms, sys: 20.9 ms, total: 253 ms
Wall time: 383 ms
